In [1]:
import dataplumbing as dp
import torch
import torchmetrics

/Users/luke/miniconda3/envs/demo/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/luke/miniconda3/envs/demo/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <870081F6-12FD-3CEA-BC5C-30F4764F2A98> /Users/luke/miniconda3/envs/demo/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <F3CB84DE-97C1-34D2-AFB8-54D9DA60ADCD> /Users/luke/miniconda3/envs/demo/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
class SelfAttentionModel(torch.nn.Module):
  def __init__(self, num_steps, num_channels, num_outputs, **kwargs):
    super().__init__(**kwargs)

    # Initialize components for self-attention
    #
    self.K = torch.nn.Parameter((2.0*torch.rand(num_channels, num_channels)-1.0)/num_channels**0.5) # Randomly intialize each weight uniformly from [ -1/num_channels**0.5, 1/num_channels**0.5 ]
    self.Q = torch.nn.Parameter((2.0*torch.rand(num_channels, num_channels)-1.0)/num_channels**0.5)
    self.V = torch.nn.Parameter((2.0*torch.rand(num_channels, num_channels)-1.0)/num_channels**0.5)

    self.softmax = torch.nn.Softmax(dim=1)

    # Initialize output layer
    #
    self.out = torch.nn.Linear(num_steps*num_channels, num_outputs)
  def forward(self, x):
    batch_size, num_steps, num_channels = x.shape
    # Run self attention
    #
    y = []
    for i in range(batch_size): # Process one sample at a time

      x_i = x[i,:,:] # x_i has shape of [ num_steps, num_channels ]

      x_k_i = torch.matmul(x_i, self.K) # x_k_i has shape of [ num_steps, num_channels ]
      x_q_i = torch.matmul(x_i, self.Q) # x_q_i has shape of [ num_steps, num_channels ]
      x_v_i = torch.matmul(x_i, self.V) # x_v_i has shape of [ num_steps, num_channels ]

      w_i = self.softmax(torch.matmul(x_q_i, x_k_i.T)/num_channels**0.5) # w_i has shape of [ num_steps, num_steps ]
      y_i = torch.matmul(w_i, x_v_i) # y_i has shape of [ num_steps, num_channels ]

      y.append(y_i)
    y = torch.stack(y, axis=0) # y has shape of [ batch_size, num_steps, num_channels ]

    # Flatten output
    #
    y_flat = y.reshape([ batch_size, num_steps*num_channels ]) # y_flat has shape of [ batch_size, num_steps*num_channels ]

    # Run output layer
    #
    l = self.out(y_flat) # l has shape of [ batch_size, num_outputs ]

    return l

In [ ]:
## 准备开始训练网络
model = SelfAttentionModel(num_steps=28**2, num_channels=1, num_outputs=10)
probability = torch.nn.Softmax(dim=1)

loss = torch.nn.CrossEntropyLoss()
accuracy = torchmetrics.classification.MulticlassAccuracy(num_classes=10)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)